### 5.1 接口

1. 接口是隐式实现的, 因此没有显示的implements关键字   
    只要一个结构体中包含接口的方法, 就自动认为该结构体的实例是某个接口的实现(当然, 要实现接口的全部代码)
2. 符号  
   从语法上看，Interface定义了一个或一组method(s)，这些method(s)只有函数签名，没有具体的实现代码`type iname interfase{ method signature() }`
    ```go
    type I interface{
        M()
    }

    type T struct {
        S string
    }

    func(t T) M() {
        fmt.Println(t.S)
    }

    func main()  {
        var i I = T{"hello"}
        i.M()
    }
    ```

3. 类型断言`(type assertion`)
    * 符号: `val.(type)`, 断言变量val的类型是type
    * 只能对实现接口的结构体进行断言

### 5.2 Error
1. Go 语言中的错误（Errors）不是异常（Exceptions）
    1. 在其它语言中，你无法确定一个函数是否会向你抛出一个异常（Exceptions）。  
        相比于抛出一个异常，Go 中的函数支持返回多个值，有一个约定俗成的用法是返回这个函数的结果并伴随一个错误（error）变量(errors are values)。  
    ```go
    func calculate(a, b int) (int, error) {
        // 一些代码
        return res, err
    }
    ```
    2. 在底层实现中，`error` 类型是一个普通的单方法接口.  
       任何拥有`Error() string`方法的结构体都默认为实现了`error`接口   
    ```go
    type error interface {
        Error() string
    }
    ```
    3. 调用者应该检查错误  
       如果一个函数由于某些原因而运行错误，则函数最后应当返回预先声明过的 error 类型变量, 这就让调用者知道发生了错误. 如果没有错误，就返回 `nil`值. 函数的调用者应当在试图使用这个返回值之前检查调用的函数是否发生了错误: 即如果 error 不是 `nil` ，函数的调用者有责任去检查这个错误并处理它（记录日志、返回错误、serve、尝试重新调用 / 清理机制等）
    ```go
    result, err := calculate(a, b)
    if err != nil {
        // 处理异常
    }
    // 继续
    ```
      在检查错误之前，结果不能被信任. Go语言确实有一个 `panic` 和 `recover` 机制，这在[另一篇 Go 博文](https://blog.golang.org/defer-panic-and-recover) 中有详细的描述。但是这不意味着去模仿异常。
      
2. 如何自定义一个err, 表示特定的错误  
   实现自己的错误类型非常容易，有非常多的方法能够让你构造实现了`Error() string`方法的结构体。任何实现了该方法的结构体, 都会被视为一个合法的错误值同时可以被返回。   
   接下来，就让我们一起去探索这些途径。 
    * [方法一]: 通过`errors.New(string)`或`fmt.Errorf(string)`产生自定义的error结构体
    ```go
	e1 := errors.New(fmt.Sprintf("Could not open file"))
	e2 := fmt.Errorf("Could not open file")

	fmt.Println(fmt.Sprintf("Type of error 1: %T", e1))
	fmt.Println(fmt.Sprintf("Type of error 2: %T", e2))

	// output:
	// Type of error 1: *errors.errorString
	// Type of error 2: *errors.errorString
    ```
    这两个方法都是go package的内置包, 

    * 模仿`os.PathError`, 自定义异常
    ```go
    type MyError struct{
        msg   string
    }
    // 意味着指针类型*MyError,才是error接口的具体类型
    func (err *MyError) Error() string{
        return err.msg
    }

    func run(s int) (string,error){
        if s==0{
            return "",&MyError{"hello error"}
        }
        return "safe",nil
    }

    func main(){
        //param := 1 // 最后打印 运行正常, 得到: safe
        param := 0   // 最后打印 发生了MyError异常: hello error
        res, err := run(param)
        if err==nil{
            fmt.Println("运行正常, 得到:",res)
            return
        }

        e,ok := err.(*MyError)  // 使用断言
        if(ok){
            fmt.Println("发生了MyError异常:",e.msg)
        }else{
            fmt.Println("发生未知异常")
        }
    }
    ```
    * 一个例子
    ```go
    for try := 0; try < 2; try++ {
        file, err = os.Create(filename)
        if err == nil {
            return
        }
        if e, ok := err.(*os.PathError); ok && e.Err == syscall.ENOSPC {
            deleteTempFiles() // Recover some space.
            continue
        }
        return
    }
    ```
   


2. panic
    * 发生panic后, 整个程序退出
    * panic表示运行时异常. 比如: 切片的index越界, 类型断言失败.
    * panic发生后, 会立刻停止当前方法的运行, 释放goroutine的栈, 如果释放后到达了goroutine的栈顶, 程序失败  
    ```go
    user := os.Getenv("USER")
    fmt.Println("user:", user)
    if user==""{
        panic("no value for $USER")
    }

    fmt.Println("program continue") // 无法执行, 发生panic后整个程序退出
    ```

3. recover  
   如果某个方法可能会抛出一个panic的异常，可以在调用这个方法之前, 定义一个defer方法, 在defer方法内部使用`recover`捕获这个异常，然后继续处理。  
   ```go
   func say() int{
        fmt.Println("a")
        fmt.Println("b")
        panic(55)
        fmt.Println("c")  // 这句无法到达, 因为上面已经发生了panic异常
        return 999
    }

    func main(){
        defer func(){
            if err:=recover(); err!=nil{
                fmt.Println("occur panic:",err)
            }
            fmt.Println("defer recover finish, exec go on .. ")
        }()
        res := say()
        fmt.Println(res)  // say()还未执行完就会发生panic异常, 调到defer处被捕获, 所以这句无法到达
    }
    //a
    //b
    //occur panic: 55
    //defer recover finish, exec go on .. 
   ```
